# 波の屈折

In [ ]:
import numpy as np
from math import cos, pi
from matplotlib import pyplot, animation, rc
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

In [ ]:
s = 1.0       # 波の伝わる速さ
L = 1.0       # 系の長さ
T = 1.2       # シミュレーション時間
n = 100       # 空間刻み数
dx = L/n      # 空間刻み幅
dt = 0.5*dx   # 時間刻み幅
m = int(T/dt) # 時間刻み数
a = s*dt/dx   # α

In [ ]:
# 初期条件
x, y = np.mgrid[0:1:dx,0:1:dx]
u0 = np.zeros((n,n)) # 初期変位
v0 = np.zeros((n,n)) # 初期速度

In [ ]:
ax = np.zeros((n,n))
ax[:,:] = a
for i in range(n):
    for j in range(n):
        if x[i,j] > 0.8*y[i,j]+0.2:
            ax[i,j] = 0.6*a

In [ ]:
# 強制振動
w = 2*pi*10 # 各振動数
def oscillate(k):
    return 0.2*cos(w*dt*k)

In [ ]:
# シミュレーション
u = np.zeros((m,n,n))
u[0,:,:] = u0[:,:]
u[0,0,:] = oscillate(0)
u[1,1:-1,1:-1] = u[0,1:-1,1:-1]+dt*v0[1:-1,1:-1] \
    +(ax[1:-1,1:-1]**2/2)*(u[0,0:-2,1:-1]+u[0,2:n,1:-1] \
    +u[0,1:-1,0:-2]+u[0,1:-1,2:n]-4*u[0,1:-1,1:-1])
u[1,0,:] = oscillate(1)
for k in range(2,m):
    u[k,1:-1,1:-1] = 2*u[k-1,1:-1,1:-1]-u[k-2,1:-1,1:-1] \
        +ax[1:-1,1:-1]**2*(u[k-1,0:-2,1:-1]+u[k-1,2:n,1:-1] \
        +u[k-1,1:-1,0:-2]+u[k-1,1:-1,2:n]-4*u[k-1,1:-1,1:-1])
    u[k,0,:] = oscillate(k)

# 結果の描画
def update(k):
    axis.clear()
    axis.plot_wireframe(x,y,u[k],linewidth=0.5)
    axis.set_zlim(-0.5,0.5)
    axis.set_title('step='+str(k))
fig = pyplot.figure()
axis = fig.add_subplot(projection='3d')
movie = animation.FuncAnimation(fig,update,frames=m,interval=50)
rc('animation', html='jshtml')
movie

In [ ]:
# 等高線プロット
def update(k):
    pyplot.cla()
    pyplot.contourf(x,y,u[k])
    pyplot.plot([0.2,1.0],[0.0,1.0],"r")
    pyplot.title('step='+str(k))
    pyplot.gca().set_aspect('equal');
fig = pyplot.figure()
movie = animation.FuncAnimation(fig,update,frames=m,interval=50)
rc('animation', html='jshtml')
movie